In [58]:
from urllib.request import Request,urlopen
import requests
from bs4 import BeautifulSoup as soup
from time import sleep
from io import StringIO
import re
from unidecode import unidecode
import os
import pybaseball as bb
import pandas as pd
import numpy as np

In [59]:
abr_convert = teams_abbr = {"AZ" : 'ARI',
    "ARI": 'ARI',
    "LAD" : 'LAD',
    "SD": 'SDP',
    "SF": 'SFG',
    "COL": 'COL',
    "HOU" : 'HOU',
    "TEX" : 'TEX',
    "SEA": 'SEA',
    "LAA": 'LAA',
    "OAK": 'OAK',
    "MIL": 'MIL',
    "CHC": 'CHC',
    'Chi. Cubs': 'CHC',
    "CIN": 'CIN',
    "PIT": 'PIT',
    "STL": 'STL',
    "MIN": 'MIN',
    "DET": 'DET',
    'Detroit': 'DET',
    "CLE": 'CLE',
    "CWS": 'CHW',
    'CHW':'CHW',
    'Chi. White Sox' : 'CHW',
    "KC": 'KCR',
    "ATL": 'ATL',
    "PHI": 'PHI',
    "MIA": 'MIA',
    'Miami': 'MIA',
    "NYM": 'NYM',
    "WSH": 'WSN',
    'Washington':'WSN',
    'WAS':'WSN',
    "BAL": 'BAL',
    "TB": 'TBR',
    "TOR": 'TOR',
    "NYY": 'NYY',
    "BOS": 'BOS',
    'Boston':'BOS'}

In [60]:
def getSoup(url):
    req = Request(url=url,headers={'User-Agent': 'Mozilla/6.0'})
    uClient = urlopen(req)
    #grabs everything from page
    html = uClient.read()
    #close connection
    uClient.close()
    #does HTML parsing
    parse = soup(html, "html.parser")
    return parse

def getCurrentDayLineup():
    #text = getSoup('https://www.mlb.com/starting-lineups')
    text = getSoup('https://www.mlb.com/starting-lineups/2024-06-27')
    away_team = text.findAll('div', {'class':"starting-lineups__team-logo-image starting-lineups__team-logo-image--away"})
    away_team = [str(x).split('data-tri-code="')[1][0:3].replace('"','') for x in away_team]
    away_team = [abr_convert.get(x) for x in away_team]

    home_team = text.findAll('div', {'class':"starting-lineups__team-logo-image starting-lineups__team-logo-image--home"})
    home_team = [str(x).split('data-tri-code="')[1][0:3].replace('"', '') for x in home_team]
    home_team = [abr_convert.get(x) for x in home_team]

    away_pitchers = text.findAll('div', {'class':'starting-lineups__pitcher-name'})
    away_pitchers = [re.search('">(.*)</a>', unidecode(str(x))).group(1) if 'TBD' not in str(x) else '' for x in away_pitchers]

    home_pitchers = away_pitchers[1::2]
    del away_pitchers[1::2]

    away_hitters = text.findAll('ol', {'class':'starting-lineups__team starting-lineups__team--away'})
    del away_hitters[1::2]
    away_hitters = [re.findall('target="">(.*)</a>', unidecode(str(x))) if 'TBD' not in str(x) else '' for x in away_hitters]

    home_hitters = text.findAll('ol', {'class':'starting-lineups__team starting-lineups__team--home'})
    del home_hitters[1::2]
    home_hitters = [re.findall('target="">(.*)</a>', unidecode(str(x))) if 'TBD' not in str(x) else '' for x in home_hitters]
    return away_hitters, home_hitters, away_pitchers, home_pitchers, away_team, home_team

def convertNames(names, team):
    ids = []
    dupe_names = {'Will Smith LAD':669257, 'Will Smith KCR': 519293}
    for name in names:
        print(name)
        if (name + ' ' + team) in dupe_names.keys():
            ids.append(dupe_names[name + ' ' + team])
        else:
            name = name.split(' ', 1)
            print(name)
            ids.append(bb.playerid_lookup(name[1], name[0], fuzzy=True).loc[0,'key_mlbam'])
    return ids

In [9]:
getCurrentDayLineup()

([['Manuel Margot',
   'Carlos Correa',
   'Royce Lewis',
   'Jose Miranda',
   'Carlos Santana',
   'Ryan Jeffers',
   'Byron Buxton',
   'Willi Castro',
   'Kyle Farmer'],
  ['Nico Hoerner',
   'Michael Busch',
   'Cody Bellinger',
   'Seiya Suzuki',
   'Ian Happ',
   'Christopher Morel',
   'Dansby Swanson',
   'Pete Crow-Armstrong',
   'Tomas Nido'],
  ['Jarred Kelenic',
   'Ozzie Albies',
   'Marcell Ozuna',
   'Matt Olson',
   'Austin Riley',
   'Sean Murphy',
   'Forrest Wall',
   'Adam Duvall',
   'Orlando Arcia'],
  ['Jazz Chisholm Jr.',
   'Bryan De La Cruz',
   'Josh Bell',
   'Jesus Sanchez',
   'Jake Burger',
   'Nick Gordon',
   'Tim Anderson',
   'Otto Lopez',
   'Ali Sanchez'],
  ['Marcus Semien',
   'Corey Seager',
   'Josh Smith',
   'Adolis Garcia',
   'Wyatt Langford',
   'Nathaniel Lowe',
   'Jonah Heim',
   'Derek Hill',
   'Leody Taveras'],
  ['Anthony Volpe',
   'Juan Soto',
   'Aaron Judge',
   'Alex Verdugo',
   'Ben Rice',
   'Jahmai Jones',
   'Austin Wells'

In [11]:
away_hitters, home_hitters, away_pitchers, home_pitchers, away_team, home_team = getCurrentDayLineup()

away_hitters_ids, away_pitchers_ids, home_hitters_ids, home_pitchers_ids = [], [], [], []
for lineup in range(len(away_hitters)):
    away_hitters_ids.append(convertNames(away_hitters[lineup], away_team[lineup]))
    away_pitchers_ids.append(convertNames([away_pitchers[lineup]], away_team[lineup]))
for lineup in range(len(home_hitters)):
    home_hitters_ids.append(convertNames(home_hitters[lineup], home_team[lineup]))
    home_pitchers_ids.append(convertNames([home_pitchers[lineup]], home_team[lineup]))

Manuel Margot
['Manuel', 'Margot']
Carlos Correa
['Carlos', 'Correa']
Royce Lewis
['Royce', 'Lewis']
Jose Miranda
['Jose', 'Miranda']
No identically matched names found! Returning the 5 most similar names.
Carlos Santana
['Carlos', 'Santana']
Ryan Jeffers
['Ryan', 'Jeffers']
Byron Buxton
['Byron', 'Buxton']
Willi Castro
['Willi', 'Castro']
Kyle Farmer
['Kyle', 'Farmer']
David Festa
['David', 'Festa']
Nico Hoerner
['Nico', 'Hoerner']
Michael Busch
['Michael', 'Busch']
Cody Bellinger
['Cody', 'Bellinger']
Seiya Suzuki
['Seiya', 'Suzuki']
Ian Happ
['Ian', 'Happ']
Christopher Morel
['Christopher', 'Morel']
Dansby Swanson
['Dansby', 'Swanson']
Pete Crow-Armstrong
['Pete', 'Crow-Armstrong']
Tomas Nido
['Tomas', 'Nido']
No identically matched names found! Returning the 5 most similar names.
Shota Imanaga
['Shota', 'Imanaga']
Jarred Kelenic
['Jarred', 'Kelenic']
Ozzie Albies
['Ozzie', 'Albies']
Marcell Ozuna
['Marcell', 'Ozuna']
Matt Olson
['Matt', 'Olson']
Austin Riley
['Austin', 'Riley']
Sea

In [2]:
### MODEL CREATION AND TESTING WITH PYBASEBALL ###

In [3]:
import pandas as pd
import numpy as np
import pybaseball as bb
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


import warnings
warnings.filterwarnings("ignore", message="The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.")
warnings.filterwarnings("ignore", message="invalid value encountered in cast")
pd.set_option('display.max_rows',20)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [4]:
### USED TO CREATE CSV THAT CONTAINS PITCHER CATEGORIES ###
def categorizePitchers():
    ### GET PITCHERS FROM DESIRED SEASONS ###
    pitchers = bb.fg_pitching_data(2023, 2024, qual=10)
    ### GET IDS FOR ALL THOSE PITCHERS ###
    pitchers_ids = bb.playerid_reverse_lookup(pitchers['IDfg'], key_type='fangraphs')

    names, mlbIDs, bbrefIDs, fgIDs, years = [], [], [], [], []
    final_stats = []
    for i in range(len(pitchers_ids)):
        print('Pitcher {0} of {1}'.format(i, len(pitchers_ids)), end='\n')
        ### GRAB DIFFERENT SEASONS FOR A PITCHER (ACCOUNT FOR THE CHANGE THEY MAY HAVE THROUGH SEASONS) ###
        start_year, end_year = int(pitchers_ids.iloc[i, 6]), int(pitchers_ids.iloc[i, 7])+1
        ### MAKE SURE THE DATA WE GET IS JUST 2021+ ###
        if start_year <= 2022: start_year = 2023
        for year in range(start_year, end_year):
            try:
                ### GRABBING SPLITS MAY ERROR, SKIP THOSE WHO DO ###
                pitcher_splits = bb.get_splits(playerid=pitchers_ids.iloc[i, 4], year=year, pitching_splits=True, player_info=True)
                ##########################################################################
                ### CATEGORIZATION BASED ON SPLITS/PITCH SPEED AND SPIN ##################
                ##########################################################################
                ### GRAB WHAT HANDEDNESS THE PITCHER IS ###
                if pitcher_splits[1]['Throws'] == 'Right ':
                    stats = np.array([0])
                elif pitcher_splits[1]['Throws'] == 'Left ':
                    stats = np.array([1])
                else:
                    stats = np.array([2])
                ### APPEND ROWS OF INTEREST TO OUR STATS ###
                stats = np.append(stats, [pitcher_splits[0].loc['Platoon Splits', 'vs LHB'].to_numpy(),
                                            pitcher_splits[0].loc['Platoon Splits', 'vs RHB'].to_numpy(),
                                            pitcher_splits[0].loc['Home or Away', 'Home'].to_numpy(),
                                            pitcher_splits[0].loc['Home or Away', 'Away'].to_numpy(),
                                            pitcher_splits[0].loc['Hit Trajectory', 'Ground Balls'].to_numpy(),
                                            pitcher_splits[0].loc['Hit Trajectory', 'Fly Balls'].to_numpy(),
                                            pitcher_splits[0].loc['Hit Trajectory', 'Line Drives'].to_numpy(),
                                            pitcher_splits[0].loc['Game Conditions', 'Day'].to_numpy(),
                                            pitcher_splits[0].loc['Game Conditions', 'Night'].to_numpy()])
                ### GET AVG SPIN AND AVG SPEED OF PITCH TYPES ###
                pitch_spin = bb.statcast_pitcher_pitch_arsenal(year=year, minP=0, arsenal_type='avg_spin')
                stats = np.append(stats, pitch_spin.loc[pitch_spin['pitcher']==pitchers_ids.iloc[i, 2], pitch_spin.columns[2:]].to_numpy())

                pitch_speed = bb.statcast_pitcher_pitch_arsenal(year=year, minP=0, arsenal_type='avg_speed')
                stats = np.append(stats, pitch_speed.loc[pitch_speed['pitcher']==pitchers_ids.iloc[i, 2], pitch_speed.columns[2:]].to_numpy())

                if len(np.nan_to_num(stats))==273:
                    ### FOR FUTURE CREATING OF PITCHER CATEGORY TABLE ###
                    final_stats.append(np.nan_to_num(stats))
                    names.append(pitchers_ids.iloc[i, 0]+', '+pitchers_ids.iloc[i, 1])
                    mlbIDs.append(pitchers_ids.iloc[i, 2])
                    bbrefIDs.append(pitchers_ids.iloc[i, 4])
                    fgIDs.append(pitchers_ids.iloc[i, 5])
                    years.append(year)
                else:
                    print('Pitcher {0} only had {1} data columns for season {2}, skipping.'.format(pitchers_ids.iloc[i,4], len(np.nan_to_num(stats)), year))
            except:
                print('Pitcher {0} encountered errors for season {1}, skipping.'.format(pitchers_ids.iloc[i, 4], year))
    final_stats = np.array(final_stats)
    kmeans = KMeans().fit(final_stats)
    pitcher_categories = pd.DataFrame({'Pitcher':names, 'Season':years, 'mlbID':mlbIDs, 'bbrefID':bbrefIDs, 'fgID':fgIDs, 'Categorization': kmeans.labels_})
    pitcher_categories.to_csv(r"C:\Users\joshm\OneDrive\Documents\Side stuff\Baseball\pitcher_categories.csv", index=False)

In [5]:
categorizePitchers()

Pitcher 0 of 715
Pitcher 1 of 715
Pitcher 2 of 715
Pitcher diazed04 encountered errors for season 2023, skipping.
Pitcher 3 of 715
Pitcher junisja01 encountered errors for season 2024, skipping.
Pitcher 4 of 715
Pitcher 5 of 715
Pitcher 6 of 715
Pitcher 7 of 715
Pitcher 8 of 715
Pitcher 9 of 715
Pitcher 10 of 715
Pitcher 11 of 715
Pitcher tylerky01 encountered errors for season 2023, skipping.
Pitcher 12 of 715
Pitcher 13 of 715
Pitcher 14 of 715
Pitcher 15 of 715
Pitcher 16 of 715
Pitcher 17 of 715
Pitcher 18 of 715
Pitcher 19 of 715
Pitcher 20 of 715
Pitcher 21 of 715
Pitcher 22 of 715
Pitcher 23 of 715
Pitcher 24 of 715
Pitcher 25 of 715
Pitcher 26 of 715
Pitcher 27 of 715
Pitcher 28 of 715
Pitcher 29 of 715
Pitcher 30 of 715
Pitcher 31 of 715
Pitcher 32 of 715
Pitcher 33 of 715
Pitcher 34 of 715
Pitcher 35 of 715
Pitcher 36 of 715
Pitcher woodfja01 encountered errors for season 2024, skipping.
Pitcher 37 of 715
Pitcher 38 of 715
Pitcher 39 of 715
Pitcher 40 of 715
Pitcher 41 of 715

TypeError: float() argument must be a string or a real number, not 'NAType'

In [ ]:
def formatGames():
    bb.cache.enable()
    games_df = bb.statcast('2023-03-25', '2024-06-20')
    games_df = games_df.dropna(subset='events')
    games_df = games_df[games_df['game_type']=='R']
    games_df = games_df[['game_date', 'game_pk', 'batter', 'pitcher', 'events', 'pitch_type', 'release_speed', 'launch_speed', 'launch_angle', 'at_bat_number']]


    for i in ['field_out', 'strikeout', 'single', 'double', 'triple', 'homerun', 'walk']:
        games_df[i] = 0
        games_df.loc[games_df['events']==i, i] = 1


    for i in ['CH', 'CU', 'FC', 'FF', 'KN', 'KC', 'SC', 'SI', 'SL', 'SV', 'FS', 'ST']:
        games_df[i] = 0
        games_df.loc[games_df['pitch_type']==i, i] = 1


    games_df = games_df.drop(['events', 'pitch_type'], axis=1)


    games_df = games_df.groupby(['game_date', 'game_pk', 'batter', 'pitcher'],as_index=False).agg({'release_speed':'mean', 'launch_speed':'mean','launch_angle':'mean' ,'at_bat_number':'nunique', 'field_out':'sum',
                                                                                  'strikeout':'sum', 'single':'sum', 'double':'sum', 'triple':'sum', 'homerun':'sum', 'walk':'sum','CH':'sum',
                                                                                  'CU':'sum', 'FC':'sum', 'FF':'sum', 'KN':'sum', 'KC':'sum', 'SC':'sum','SI':'sum','SL':'sum',
                                                                                  'SV':'sum','FS':'sum','ST':'sum'})
    return games_df

def getBatterPitcherMatchup(batter, pitcher, games, pitcher_categories):
    matchup = games[(games['batter']==batter)&(games['pitcher']==pitcher)]
    at_bats = np.sum(games['at_bats'])
    if at_bats < 15:
        category = pitcher_categories.loc[pitcher_categories['mlbIDs']==pitcher, 'Categorization'].to_list()[-1]
        games = pd.merge(games, pitcher_categories, how='left', left_on='mlbIDs', right_on='pitcher')
        extra_matchups = games[(games['batter']==batter)&(games['pitcher']!=pitcher)&(games['Categorization']==category)]
        extra_matchups = extra_matchups.sort_values('game_date', 'descending')
        while (at_bats < 15) & (len(extra_matchups) > 0):
            matchup = pd.concat([matchup, extra_matchups.iloc[0, :]])
            at_bats += extra_matchups.iloc[0, 8]
            extra_matchups = extra_matchups.drop(index=0)
    matchup = matchup[['batter', 'release_speed', 'launch_speed', 'launch_angle', 'field_out', 'strikeout', 'single', 'double', 'triple', 'homerun', 'walk', \
                       'CH', 'CU', 'FC', 'FF', 'KN', 'KC', 'SC', 'SI', 'SL', 'SV', 'FS', 'ST']].groupby('batter').mean()
    stat_array = matchup.iloc[batter].to_numpy()
    return stat_array

def trainModel():
    main_df, pitcher_categories = formatGames(), pd.read_csv('blank', usecols=['mlbIDs', 'Categorization']) #Input file with pitcher IDs and categories here
    game_ids = pd.unique(main_df['game_pk']).to_list()
    train, test = main_df[main_df['game_pk'] <= game_ids[np.ceil(len(game_ids)*0.8)]], main_df[main_df['game_pk']>game_ids[np.ceil(len(game_ids)*0.8)]]
    input_batter, output_batter, batter_names = [], [], []
    for game in set(train['game_pk']):
        temp = train[train['game_pk']<game]
        curr_batters = set(train.loc[train['game_pk']==game, 'batter'])
        for batter in curr_batters:
            batter_names.append(batter)
            pitcher = train.loc[(train['game_pk']==game)&(train['batter']==batter), 'pitcher']
            input_batter.append(getBatterPitcherMatchup(batter=batter, pitcher=pitcher, games=temp, pitcher_categories=pitcher_categories))
            ### GET CURRENT GAME STATS ###
            curr_stats = train.loc[(train['game_pk']==game)&(train['batter']==batter), ['release_speed', 'launch_speed', 'launch_angle', 'field_out', 'strikeout', 'single', 'double', 'triple', 'homerun', 'walk', \
                       'CH', 'CU', 'FC', 'FF', 'KN', 'KC', 'SC', 'SI', 'SL', 'SV', 'FS', 'ST']].to_numpy()
            output_batter.append(curr_stats)

In [55]:
test = bb.team_game_logs(2024, 'ARI', 'pitching')

In [56]:
test

,Game,Date,Home,Opp,Rslt,IP,H,R,ER,UER,...,2B,3B,IBB,SH,SF,ROE,GDP,NumPlayers,Umpire,PitchersUsed
0,1,Mar 28,True,COL,"W,16-1",9.0,4,1,1,0,...,2,0,0,0,0,0,1,5,Dan Bellino,"Z.Gallen (99-58-W), S.McGough (99), M.Castro (..."
1,2,Mar 29,True,COL,"W,7-3",9.0,7,3,3,0,...,1,1,0,0,0,0,0,4,Phil Cuzzi,"M.Kelly (99-72-W), J.Mantiply (99), R.Thompson..."
2,3,Mar 30,True,COL,"L,4-9",9.0,10,9,9,0,...,1,1,0,0,1,0,2,4,Tony Randazzo,"T.Henry (99-32-L), L.Frías (1), B.Jarvis (99),..."
3,4,Mar 31,True,COL,"W,5-1",9.0,8,1,1,0,...,0,0,0,0,0,0,2,4,Alex Tosi,"B.Pfaadt (99-59-W), S.McGough (2), R.Thompson ..."
4,5,Apr 1,True,NYY,"L,2-5",9.0,8,5,4,1,...,3,0,0,0,1,0,1,5,Larry Vanover,"R.Nelson (99-28-L), K.Nelson (1), M.Castro (3)..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,158,Sep 24,True,SFG,"L,0-11",9.0,14,11,8,3,...,2,0,0,0,0,1,2,4,Lance Barrett,"B.Pfaadt (4-30-L), B.Walston (0), Y.Diaz (1), ..."
158,159,Sep 25,True,SFG,"W,8-2",9.0,6,2,2,0,...,3,0,0,0,0,0,0,4,Roberto Ortiz,"Z.Gallen (4-73-W), J.Martinez (2), A.Puk (3-H)..."
159,160,Sep 27,True,SDP,"L,3-5",9.0,10,5,5,0,...,4,1,0,0,0,0,0,5,Alex Tosi,"M.Kelly (5-40-L), J.Mantiply (4), R.Thompson (..."
160,161,Sep 28,True,SDP,"L,0-5",9.0,10,5,5,0,...,0,0,0,0,0,0,0,7,Jordan Baker,"E.Rodríguez (4-57), K.Ginkel (0), J.Mantiply (..."


In [61]:
odds = pd.read_csv(r"C:\Users\joshm\OneDrive\Documents\Side stuff\Baseball\odds_list.csv")
odds['Home Team'], odds['Away Team'] = odds['Home Team'].apply(lambda x: teams_abbr[x]), odds['Away Team'].apply(lambda x: teams_abbr[x])
odds['Date'] = pd.to_datetime(odds['Date'])
odds = odds.loc[odds['Date'].dt.year>=2023]
odds

,Date,Home Team,Home W Odds,Away Team,Away W Odds
4967,2023-03-30,NYY,1.57,SFG,2.45
4968,2023-03-30,WSN,3.00,ATL,1.40
4969,2023-03-30,BOS,1.74,BAL,2.10
4970,2023-03-30,CHC,2.15,MIL,1.71
4971,2023-03-30,TBR,1.44,DET,2.85
...,...,...,...,...,...
9908,2024-09-29,COL,2.15,LAD,1.74
9909,2024-09-29,ATL,1.40,KCR,3.00
9910,2024-09-29,CHC,2.00,CIN,1.83
9911,2024-09-30,ATL,1.67,NYM,2.25


In [62]:
all_teams = pd.unique(odds['Home Team'])
all_team_gamelogs_batting = pd.DataFrame()

for season in [2023, 2024]:
    for team in all_teams:
        gamelog_batting = bb.team_game_logs(season, team, 'batting')
        gamelog_batting['Date'] = pd.to_datetime(gamelog_batting['Date'].str[:6].str.strip() + ', {0}'.format(season))
        gamelog_batting['OppStart'] = gamelog_batting['OppStart'].str.split('(').str[0]
        all_team_gamelogs_batting = pd.concat([all_team_gamelogs_batting, gamelog_batting.drop('Thr', axis=1)])
        
all_team_gamelogs_batting.to_csv(r"C:\Users\joshm\OneDrive\Documents\Side stuff\Baseball\team_gamelogs.csv", index=False)

c:\Users\joshm\AppData\Local\Programs\Python\Python310\lib\site-packages\pybaseball\team_game_logs.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]
c:\Users\joshm\AppData\Local\Programs\Python\Python310\lib\site-packages\pybaseball\team_game_logs.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]
c:\Users\joshm\AppData\Local\Programs\Python\Python310\lib\site-packages\pybaseball\team_game_logs.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]
c:\Users\joshm\AppData\Local\Programs\Python\Python310\lib\site-

In [32]:
pd.to_datetime('Mar 28, 2023')

Timestamp('2023-03-28 00:00:00')